In [1]:
import pandas as pd

In [7]:
# now that we have the df for twitter sentiment of each day fron 20170101
sentiment_filename = 'dailysentiments.csv'
stock_filename = 'stockprice_raw.csv'
sentiments = pd.read_csv(sentiment_filename, engine='python',index_col='date')
stocks = pd.read_csv(stock_filename, engine = 'python',index_col='Date')
sentiments = sentiments.drop(columns=sentiments.columns[0])

In [8]:
# set the size of the sliding window of stock price prediction
window_size = 1
# use Open_prev to find the opening price of the previous day
stocks['Open_prev'] = stocks['Open'].shift(window_size)

In [10]:
from datetime import timedelta
dates = stocks.index
dates = pd.to_datetime(dates)

#to correct for if the gap is more than one day
for k in range(stocks.shape[0]):
  if k > 0 and (dates[k] - dates[k-1]) > timedelta(days =1):
    idx = stocks.index[k]
    stocks.at[idx, 'Open_prev'] = stocks.iloc[k-1]['Open']

In [11]:
stocks

,date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Open_prev
Date,,,,,,,,,
2016-12-30,20161230,43.259998,43.500000,42.335999,42.737999,23213000,0,0.0,NaN
2017-01-03,20170103,42.972000,44.066002,42.192001,43.397999,29616500,0,0.0,43.259998
2017-01-04,20170104,42.950001,45.599998,42.862000,45.397999,56067500,0,0.0,42.972000
2017-01-05,20170105,45.284000,45.495998,44.389999,45.349998,29558500,0,0.0,42.950001
2017-01-06,20170106,45.386002,46.062000,45.090000,45.801998,27639500,0,0.0,45.284000
...,...,...,...,...,...,...,...,...,...
2020-12-28,20201228,674.510010,681.400024,660.799988,663.690002,32278600,0,0.0,642.989990
2020-12-29,20201229,661.000000,669.900024,655.000000,665.989990,22910800,0,0.0,674.510010
2020-12-30,20201230,672.000000,696.599976,668.359985,694.780029,42846000,0,0.0,661.000000


In [12]:
stocks['Diff'] = stocks['Close'] - stocks['Open_prev']
stocks.loc[stocks['Diff']> 0, 'Diff'] = 1
stocks.loc[stocks['Diff']< 0, 'Diff'] = -1
stocks.loc[stocks['Diff'] == 0, 'Diff'] = 0
stock_label_df = stocks[['date','Diff']]

In [13]:
stock_label_df
#stock_label_df.to_csv('stockprice_label.csv')

,date,Diff
Date,,
2016-12-30,20161230,NaN
2017-01-03,20170103,1.0
2017-01-04,20170104,1.0
2017-01-05,20170105,1.0
2017-01-06,20170106,1.0
...,...,...
2020-12-28,20201228,1.0
2020-12-29,20201229,-1.0
2020-12-30,20201230,1.0


In [15]:
'''
previous code:
# Create a df with sentiment analysis from previous two days given the dateindex 
sentiment_prev2days = pd.concat([sentiments.shift(1),sentiments.shift(2)],axis=1)
sentiment_prev2days = sentiment_prev2days.dropna()
sentiment_prev2days.insert(0,'date', sentiment_prev2days.index)
sentiment_prev2days.index.names = ['Date']
sentiment_prev2days
# note that 1448 tweet days vs. 1007 valid stock labels

'''

#sentiment analysis window
sentiment_window_size = 7

#first shift to the day before stock price window
sentiment_shift = sentiments.shift(window_size+1)

#accumulate features of sentiment analysis window
for i in range(3, sentiment_window_size + 2):
  sentiment_shift = pd.concat([sentiment_shift,sentiments.shift(i)],axis=1)

sentiment_shift = sentiment_shift.dropna()

sentiment_shift

,pos_mean,pos_sd,neu_mean,neu_sd,neg_mean,neg_sd,pos_mean,pos_sd,neu_mean,neu_sd,...,neu_mean,neu_sd,neg_mean,neg_sd,pos_mean,pos_sd,neu_mean,neu_sd,neg_mean,neg_sd
date,,,,,,,,,,,,,,,,,,,,,
20170109,0.199510,0.216766,0.725545,0.208317,0.074946,0.117283,0.313490,0.229368,0.585565,0.234168,...,0.526066,0.270588,0.068304,0.100616,0.389289,0.278815,0.550918,0.271281,0.059792,0.132426
20170110,0.466444,0.321560,0.524183,0.315386,0.009374,0.012908,0.199510,0.216766,0.725545,0.208317,...,0.369249,0.198773,0.426441,0.289451,0.405631,0.297854,0.526066,0.270588,0.068304,0.100616
20170111,0.344037,0.363655,0.315183,0.259858,0.340780,0.398341,0.466444,0.321560,0.524183,0.315386,...,0.620133,0.230882,0.119219,0.241072,0.204310,0.305864,0.369249,0.198773,0.426441,0.289451
20170112,0.277461,0.244450,0.636482,0.253647,0.086058,0.209689,0.344037,0.363655,0.315183,0.259858,...,0.522812,0.168254,0.079312,0.167570,0.260648,0.209138,0.620133,0.230882,0.119219,0.241072
20170113,0.417612,0.170505,0.556058,0.149221,0.026331,0.059264,0.277461,0.244450,0.636482,0.253647,...,0.585565,0.234168,0.100946,0.230763,0.397876,0.232893,0.522812,0.168254,0.079312,0.167570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201227,0.438897,0.367852,0.421036,0.305266,0.140067,0.232719,0.446829,0.357667,0.452923,0.314052,...,0.519981,0.312640,0.076255,0.168727,0.398923,0.346938,0.425238,0.277643,0.175839,0.252596
20201228,0.371328,0.354282,0.497004,0.325460,0.131668,0.243043,0.438897,0.367852,0.421036,0.305266,...,0.410210,0.284745,0.146249,0.241538,0.403763,0.339747,0.519981,0.312640,0.076255,0.168727
20201229,0.495777,0.365542,0.399551,0.304094,0.104671,0.213257,0.371328,0.354282,0.497004,0.325460,...,0.572851,0.286128,0.195474,0.271481,0.443541,0.358427,0.410210,0.284745,0.146249,0.241538


In [18]:
dataframe = sentiment_shift.merge(stock_label_df,how='left', left_on='date', right_on='date')
dataframe = dataframe.dropna()
dataframe

,date,pos_mean,pos_sd,neu_mean,neu_sd,neg_mean,neg_sd,pos_mean,pos_sd,neu_mean,...,neu_sd,neg_mean,neg_sd,pos_mean,pos_sd,neu_mean,neu_sd,neg_mean,neg_sd,Diff
0,20170109,0.199510,0.216766,0.725545,0.208317,0.074946,0.117283,0.313490,0.229368,0.585565,...,0.270588,0.068304,0.100616,0.389289,0.278815,0.550918,0.271281,0.059792,0.132426,1.0
1,20170110,0.466444,0.321560,0.524183,0.315386,0.009374,0.012908,0.199510,0.216766,0.725545,...,0.198773,0.426441,0.289451,0.405631,0.297854,0.526066,0.270588,0.068304,0.100616,1.0
2,20170111,0.344037,0.363655,0.315183,0.259858,0.340780,0.398341,0.466444,0.321560,0.524183,...,0.230882,0.119219,0.241072,0.204310,0.305864,0.369249,0.198773,0.426441,0.289451,-1.0
3,20170112,0.277461,0.244450,0.636482,0.253647,0.086058,0.209689,0.344037,0.363655,0.315183,...,0.168254,0.079312,0.167570,0.260648,0.209138,0.620133,0.230882,0.119219,0.241072,1.0
4,20170113,0.417612,0.170505,0.556058,0.149221,0.026331,0.059264,0.277461,0.244450,0.636482,...,0.234168,0.100946,0.230763,0.397876,0.232893,0.522812,0.168254,0.079312,0.167570,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,20201224,0.231675,0.267264,0.572851,0.286128,0.195474,0.271481,0.443541,0.358427,0.410210,...,0.317613,0.134209,0.257586,0.311565,0.327600,0.540178,0.292977,0.148257,0.222403,1.0
1423,20201228,0.371328,0.354282,0.497004,0.325460,0.131668,0.243043,0.438897,0.367852,0.421036,...,0.284745,0.146249,0.241538,0.403763,0.339747,0.519981,0.312640,0.076255,0.168727,1.0
1424,20201229,0.495777,0.365542,0.399551,0.304094,0.104671,0.213257,0.371328,0.354282,0.497004,...,0.286128,0.195474,0.271481,0.443541,0.358427,0.410210,0.284745,0.146249,0.241538,-1.0
1425,20201230,0.408934,0.347921,0.457482,0.300991,0.133583,0.246821,0.495777,0.365542,0.399551,...,0.304817,0.137060,0.231174,0.231675,0.267264,0.572851,0.286128,0.195474,0.271481,1.0


In [21]:

filename = "database_prev{}days.csv".format(sentiment_window_size)
filename

'database_prev7days.csv'

In [22]:
dataframe.to_csv(filename)